# 🛠️ Notebook 02: Web Search and Agentic Reasoning with the Responses API

In this notebook you’ll see how to:

- Enable the built-in **`web_search`** tool in the Responses API  
- Let the model decide when and how to call that tool  
- Prompt the model to **show its reasoning** (plan + justification)  
- Generate a final artifact (an email invitation) based on that plan

Instead of wiring up custom Python tools, we use OpenAI’s hosted `web_search`
tool plus good instructions to get agent-like behaviour:

1. **Gather information** using web search  
2. **Evaluate options** (which concert is best?)  
3. **Explain the reasoning** in a dedicated section  
4. **Act** by drafting a ready-to-send email

## What you'll build

- a simple call that uses `web_search` to find concerts near a location  
- a combined “search + email” workflow in a single Responses call  
- a structured output that separates **Reasoning** from **Final email**

## Prerequisites

1. `OPENAI_API_KEY` in your `.env` at the repo root  
2. `pip install -r requirements.txt`  
3. Run this notebook from the repo root so `utils/` imports resolve

In [4]:
from __future__ import annotations

import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

# --- Ensure project root is on sys.path ---
PROJECT_ROOT = Path("..").resolve()  # parent of notebooks/
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

from utils.openai_client import get_response  # still useful

# Load env + create client
load_dotenv(PROJECT_ROOT / ".env")
client = OpenAI()

PROJECT_ROOT: C:\Users\user\Desktop\OpenAI-responses-api-hub


In [5]:
prompt = """
You are a helpful weekend planner.

Use web_search to find 3 live music concerts happening this weekend
near Tampa, Florida. Return them in a short bullet list with:

- artist or event name
- date
- venue
- city
"""

response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt,
    tools=[{"type": "web_search"}],
)

print(response.output_text)

Here are three live music concerts happening near Tampa, Florida, this weekend:

- **Daily Bread**
  - **Date:** Saturday, November 22, 2025
  - **Venue:** Jannus Live
  - **City:** St. Petersburg, FL

- **Taimane**
  - **Date:** Sunday, November 23, 2025
  - **Venue:** David A. Straz Center - Ferguson Hall
  - **City:** Tampa, FL

- **Florida Orchestra: Mozart & Handel**
  - **Date:** Sunday, November 23, 2025
  - **Venue:** Mahaffey Theater at the Duke Energy Center for the Arts
  - **City:** St. Petersburg, FL

Enjoy the concerts! 


## 3. Agentic reasoning: plan *then* act

Right now the model quietly calls `web_search` and jumps straight to the final email.

In many real apps we want **two things**:

1. A short, human-readable **reasoning trace** (how it picked the event), and  
2. The final artifact (the email).

We can get that by tightening the prompt: ask the model to show a structured
“Reasoning” section first, then the final email.

In [8]:
reasoned_prompt = """
You are my weekend concierge and thoughtful planner.

Your job:

1. Use web_search to find at least two live music concerts happening
   this weekend near Tampa, Florida.
2. Decide which one is the best option for a casual night out with a friend.
3. Explain your decision in a short 'Reasoning' section.
4. Then write the final email invitation.

Return your answer in **this exact structure**:

Reasoning:
- (one or more bullet points explaining what you searched for,
  what you found, and why you chose this event)

Final email:
(email body here, ready to paste into Gmail)
"""

reasoned_response = client.responses.create(
    model="gpt-4.1-mini",
    input=reasoned_prompt,
    tools=[{"type": "web_search"}],
)

print(reasoned_response.output_text)

Reasoning:
- I searched for live music concerts happening near Tampa, Florida, on November 22–23, 2025.
- I found several events, including:
  - **"Cigars n' Soul" featuring Live Saxophonist** on Saturday, November 22, at 9:00 PM at 10660 Palm River Rd. ([allevents.in](https://allevents.in/tampa/music?utm_source=openai))
  - **"Cigars n' Soul" featuring Live Saxophonist** on Saturday, November 29, at 9:00 PM at 10660 Palm River Rd. ([allevents.in](https://allevents.in/tampa/music?utm_source=openai))
  - **"How Sweet It Is - James Taylor Tribute"** on Saturday, November 22, at 7:00 PM at Pinellas Park Performing Arts Center. ([tampa-bay.events](https://tampa-bay.events/concerts/november/?utm_source=openai))
  - **"Easy Honey"** on Saturday, November 22, at 7:00 PM at The Crowbar. ([tampa-bay.events](https://tampa-bay.events/concerts/november/?utm_source=openai))
- Considering the options, the **"Cigars n' Soul" featuring Live Saxophonist** event on Saturday, November 22, at 9:00 PM at 1

---

## 🔧 Advanced (Optional): Custom Calculator Tool

So far, we’ve only used a **built-in tool** (`web_search`) that OpenAI runs for us.

In this section, you’ll see how to:

- Define a **custom function tool** (`basic_calculator`)
- Pass its **JSON schema** via `tools=[...]`
- Inspect the model’s **tool call output** to see how it would invoke your function

This is a stepping stone toward building full agents where your own Python
functions (APIs, databases, etc.) are exposed as tools the model can call.

In [9]:
from typing import Any, Dict, List
import json

def basic_calculator(operation: str, a: float, b: float) -> float:
    """Deterministic math helper that supports add, subtract, multiply, and divide."""
    if operation == "add":
        return a + b
    if operation == "subtract":
        return a - b
    if operation == "multiply":
        return a * b
    if operation == "divide":
        if b == 0:
            raise ValueError("Cannot divide by zero.")
        return a / b
    raise ValueError(f"Unsupported operation: {operation!r}")

CALC_TOOLS: List[Dict[str, Any]] = [
    {
        "type": "function",
        "name": "basic_calculator",
        "description": "Perform deterministic arithmetic (add, subtract, multiply, divide).",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "Math operation to execute.",
                },
                "a": {
                    "type": "number",
                    "description": "First operand.",
                },
                "b": {
                    "type": "number",
                    "description": "Second operand.",
                },
            },
            "required": ["operation", "a", "b"],
            "additionalProperties": False,
        },
    }
]

### Under the hood (optional): raw tool call objects

This section is mostly for developers. It shows the exact `function_call`
objects the Responses API returns when it plans to use `basic_calculator`.
Feel free to collapse this cell if you just want the high-level view.

In [10]:
calc_prompt = """
You are a careful math assistant.

You have access to a function tool called basic_calculator.
You MUST use this tool for all arithmetic instead of doing math yourself.

Tasks:
1. Compute 13.5 * 1.2
2. Compute 42 - 17
3. Compute 5 + 7

First, call the tool once for each calculation.
Then, after the tool calls, explain the results in plain English.
"""

calc_response = client.responses.create(
    model="gpt-4.1-mini",
    input=calc_prompt,
    tools=CALC_TOOLS,
)

print("Raw output items:\n")
for idx, item in enumerate(calc_response.output):
    print(f"--- output[{idx}] (type={item.type}) ---")
    print(item.model_dump_json(indent=2))

Raw output items:

--- output[0] (type=function_call) ---
{
  "arguments": "{\"operation\":\"multiply\",\"a\":13.5,\"b\":1.2}",
  "call_id": "call_fZh2UhzYcgWueMiwWHGGkfWJ",
  "name": "basic_calculator",
  "type": "function_call",
  "id": "fc_0f1e9ec34c6f55a400691a7856946081a19e606a8ff588bea4",
  "status": "completed"
}
--- output[1] (type=function_call) ---
{
  "arguments": "{\"operation\":\"subtract\",\"a\":42,\"b\":17}",
  "call_id": "call_gWmllry5B5OrtfI1Cj3ULzeG",
  "name": "basic_calculator",
  "type": "function_call",
  "id": "fc_0f1e9ec34c6f55a400691a7856ab4081a188a435b83de6ed16",
  "status": "completed"
}
--- output[2] (type=function_call) ---
{
  "arguments": "{\"operation\":\"add\",\"a\":5,\"b\":7}",
  "call_id": "call_SMqz7aLl430wOQzezS5JvInr",
  "name": "basic_calculator",
  "type": "function_call",
  "id": "fc_0f1e9ec34c6f55a400691a785717d081a1a850c793ba9497d1",
  "status": "completed"
}


### Friendly view: what the calculator actually did

Below we translate the tool calls into a simple table and bullet list so you
can see, in plain English, what the model asked the calculator to do and what
results it got.

In [13]:
import pandas as pd
import json

# Map operation names to symbols for nicer display
OP_SYMBOLS = {
    "add": "+",
    "subtract": "-",
    "multiply": "×",
    "divide": "÷",
}

rows = []

for item in calc_response.output:
    # We only care about the calculator function calls
    if getattr(item, "type", "") != "function_call":
        continue

    args = json.loads(item.arguments or "{}")
    op = args.get("operation")
    a = args.get("a")
    b = args.get("b")

    # Use our Python function to get the true result
    try:
        result = basic_calculator(operation=op, a=a, b=b)
    except Exception as e:
        result = f"Error: {e}"

    rows.append(
        {
            "Expression": f"{a} {OP_SYMBOLS.get(op, '?')} {b}",
            "Operation": op,
            "a": a,
            "b": b,
            "Result": result,
        }
    )

# 1) Spreadsheet-style view
calc_df = pd.DataFrame(rows)
display(calc_df)

# 2) Plain-language summary for non-technical readers
print("\nSummary:")
for row in rows:
    print(f"- {row['Expression']} = {row['Result']}")

,Expression,Operation,a,b,Result
0,13.5 × 1.2,multiply,13.5,1.2,16.2
1,42 - 17,subtract,42.0,17.0,25.0
2,5 + 7,add,5.0,7.0,12.0



Summary:
- 13.5 × 1.2 = 16.2
- 42 - 17 = 25
- 5 + 7 = 12


In this advanced section, the model didn’t just “do math in its head” — it
actually **called a real Python calculator**, and we inspected those calls and
results. That’s the same pattern you can use to connect the model to your own
APIs, databases, or business logic.